In [1]:
import os

In [2]:
%pwd

'f:\\Masroom\\mushroom_classification\\test'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\Masroom\\mushroom_classification'

Crate the entity

In [5]:
# Crate the entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

update the configuration manager in src config

In [7]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

update the components

In [9]:
import os
import urllib.request as request
import zipfile
from src import logger
from src.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(
                f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {
                        get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

update the pipeline

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-20 01:04:41,388: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-20 01:04:41,684: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-20 01:04:41,874: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-20 01:04:41,876: INFO: common: created directory at: artifacts]
[2024-04-20 01:04:41,877: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-20 01:04:44,454: INFO: 1268487314: artifacts/data_ingestion/mushrooms.zip download! with following info: 
Connection: close
Content-Length: 27586
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3c4f7b776333f24077599c95d3fb073e2118351a0fdd11623682b96e4e38cf75"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6352:28D757:F30B0:13FE6B:6622C695
Accept-Ranges: bytes
Date: Fri